In [1]:
from os import environ, makedirs

input_folder = environ.get('CROSSCOMPUTE_INPUT_FOLDER', 'tests/standard/input')
output_folder = environ.get('CROSSCOMPUTE_OUTPUT_FOLDER', 'tests/standard/output')
try:
    makedirs(output_folder)
except IOError:
    pass

In [2]:
import json
from os.path import join

settings_path = join(input_folder, 'settings.json')
d = json.load(open(settings_path, 'rt'))
d

{'name': 'USA_MO_Springfield.Rgnl.AP'}

In [3]:
raw_place_name = d['name']
place_name = d['name'].replace('-', ' ').replace('_', ' ').replace('.', ' ')
place_name

'USA MO Springfield Rgnl AP'

In [4]:
json.dump({
    'place_name': place_name,
}, open(join(output_folder, 'settings.json'), 'wt'))

In [5]:
open(join(output_folder, 'description.md'), 'wt').write(f'''
Here are residential energy load profiles for **{place_name}** based on simulated data from
[Building America House Simulation Protocols](http://www.nrel.gov/docs/fy11osti/49246.pdf). 
Raw data is available [here](https://openei.org/doe-opendata/dataset/commercial-and-residential-hourly-load-profiles-for-all-tmy3-locations-in-the-united-states).
'''.strip())

361

In [6]:
datasets_folder = 'datasets'
residential_folder = join(datasets_folder, 'RESIDENTIAL_LOAD_DATA_E_PLUS_OUTPUT')

In [7]:
import sys

def add_modules_folder(folder):
    if folder not in sys.path:
        sys.path.append(folder)

add_modules_folder('../../modules')

In [8]:
from routines import (
    load_table,
    parse_timestamp,
    save_date_plot,
    save_table_description,
    save_table_plot,
    split_table)

In [9]:
from datetime import datetime
from glob import glob

customer_type = 'Residential'
level_categories = 'BASE', 'LOW', 'HIGH'

for level_category in level_categories:
    table_path = glob(join(
        residential_folder, level_category, raw_place_name + '.*'))[0]
    level_category = level_category.capitalize()    
    t = load_table(table_path)
    electricity_table, gas_table = split_table(t)
    energy_packs = [
        ('Electricity', electricity_table),
        ('Gas', gas_table),
    ]
    for energy_type, energy_table in energy_packs:
        save_table_description(
            output_folder, energy_table, customer_type,
            energy_type, level_category)
        save_table_plot(
            output_folder, energy_table, customer_type,
            energy_type, level_category, place_name)
        save_date_plot(
            output_folder, '2000-01-01', '2000-01-02',
            energy_table, customer_type,
            energy_type, level_category, place_name)
        save_date_plot(
            output_folder, '2000-06-01', '2000-06-02',
            energy_table, customer_type,
            energy_type, level_category, place_name)

In [14]:
t.describe(percentiles=[]).drop(['count', 'std', '50%'])

,Electricity:Facility,Gas:Facility,Heating:Electricity,Heating:Gas,Cooling:Electricity,HVACFan:Fans:Electricity,Electricity:HVAC,Fans:Electricity,General:InteriorLights:Electricity,General:ExteriorLights:Electricity,Appl:InteriorEquipment:Electricity,Misc:InteriorEquipment:Electricity,Water Heater:WaterSystems:Electricity
mean,2.155709,3.563467,0.0,3.530211,0.392139,0.179647,0.571786,0.179647,0.314701,0.071135,0.302307,0.482062,0.388136
min,0.598001,0.022526,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.075846,0.017144,0.066957,0.326520,0.034784
max,6.966891,21.296823,0.0,21.255754,4.562356,0.756987,5.319343,0.756987,1.281159,0.289593,0.584191,0.828492,1.022540
